<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- KT4 파일로 500명 샘플링해서 사용자 로그 quit 횟수, timestamp 차이(소요시간), mobile/web 비율, submit 비율 계산함
- 500명 행동 패턴 통계 출력

✅ 내부 CSV 파일 87861개 탐색됨


📦 압축 해제 중:  42%|████▏     | 210/500 [03:33<04:36,  1.05it/s]

-- =============================================
-- 🎯 EdNet-KT4 학습행동 분석 (AWS Athena SQL)
-- =============================================

WITH
-- --------------------------------------------------
-- ① 기본 통계
-- --------------------------------------------------
base AS (
  SELECT
    user_id,
    COUNT_IF(event_type = 'respond') AS n_attempts,
    COUNT_IF(event_type = 'submit') AS n_submit,
    COUNT_IF(event_type = 'quit') AS n_quit,
    COUNT(DISTINCT problem_id) AS n_questions,
    COUNT(DISTINCT bundle_id) AS n_bundles,
    COUNT(DISTINCT DATE(timestamp)) AS active_days
  FROM kt4_logs
  GROUP BY user_id
),

-- --------------------------------------------------
-- ② 시간 기반 반응 지표
-- --------------------------------------------------
rt_stats AS (
  SELECT
    user_id,
    AVG(diff_sec) AS rt_mean,
    stddev_samp(diff_sec) AS rt_std,
    stddev_samp(EXTRACT(hour FROM timestamp)) AS consistent_study_rhythm,
    stddev_samp(diff_sec) / NULLIF(AVG(diff_sec), 0) AS rt_cv
  FROM (
    SELECT
      user_id,
      timestamp,
      CAST(
        date_diff('second',
          LAG(timestamp) OVER (PARTITION BY user_id ORDER BY timestamp),
          timestamp
        ) AS DOUBLE
      ) AS diff_sec
    FROM kt4_logs
    WHERE event_type = 'respond'
  )
  GROUP BY user_id
),

-- --------------------------------------------------
-- ③ 선택·행동 패턴
-- --------------------------------------------------
behavior AS (
  SELECT
    user_id,
    SUM(choice_changed) AS n_choice_changes,
    SUM(IF(event_type IN ('erase_choice','undo_erase_choice'),1,0)) AS erasure_used,
    SUM(IF(event_type IN ('erase_choice','undo_erase_choice'),1,0)) * 1.0 /
      NULLIF(COUNT_IF(event_type='respond'),0) AS erasure_used_rate,
    SUM(IF(event_type LIKE '%audio%' OR event_type LIKE '%video%',1,0)) AS media_toggle_count
  FROM kt4_logs
  GROUP BY user_id
),

-- --------------------------------------------------
-- ④ quit 관련 지표
-- --------------------------------------------------
quit_info AS (
  SELECT
    user_id,
    COUNT_IF(event_type='quit') AS quit_cnt,
    COUNT_IF(event_type='submit') AS submit_cnt,
    (COUNT_IF(event_type='quit') - COUNT_IF(event_type='submit')) * 1.0 /
      NULLIF(COUNT_IF(event_type='quit'),0) AS quit_without_submit_rate
  FROM kt4_logs
  GROUP BY user_id
),

-- --------------------------------------------------
-- ⑤ 정답률 관련 지표
-- --------------------------------------------------
accuracy AS (
  SELECT
    user_id,
    AVG(first_correct) AS first_try_accuracy,
    AVG(last_correct) AS accuracy_final,
    AVG(IF(first_correct=0 AND last_correct=1,1,0)) AS self_correction_rate,
    AVG(IF(first_correct=1 AND last_correct=0,1,0)) AS regret_rate,
    (AVG(last_correct) - AVG(first_correct)) AS improvement_delta
  FROM (
    SELECT
      user_id, problem_id,
      element_at(split(array_join(array_agg(CAST(correct AS VARCHAR) ORDER BY timestamp ASC),','),','),1) AS first_correct,
      element_at(split(array_join(array_agg(CAST(correct AS VARCHAR) ORDER BY timestamp DESC),','),','),1) AS last_correct
    FROM kt4_logs
    WHERE event_type='respond'
    GROUP BY user_id, problem_id
  )
  GROUP BY user_id
),

-- --------------------------------------------------
-- ⑥ 학습 리듬 및 플랫폼 비율
-- --------------------------------------------------
rhythm AS (
  SELECT
    user_id,
    AVG(diff_sec)/60 AS study_minutes_day,
    stddev_samp(EXTRACT(hour FROM timestamp)) AS start_hour_std,
    AVG(IF(platform='mobile',1,0)) AS platform_ratio_mobile,
    AVG(IF(platform='web',1,0)) AS platform_ratio_web
  FROM (
    SELECT
      user_id,
      timestamp,
      date_diff('second',
        LAG(timestamp) OVER (PARTITION BY user_id ORDER BY timestamp),
        timestamp
      ) AS diff_sec
    FROM kt4_logs
  )
  GROUP BY user_id
),

-- --------------------------------------------------
-- ⑦ Z-score 정규화 (전체 사용자 단위)
-- --------------------------------------------------
z_norm AS (
  SELECT
    user_id,
    (n_choice_changes - AVG(n_choice_changes) OVER()) / NULLIF(stddev_samp(n_choice_changes) OVER(),0) AS z_n_choice_changes,
    (erasure_used_rate - AVG(erasure_used_rate) OVER()) / NULLIF(stddev_samp(erasure_used_rate) OVER(),0) AS z_erasure_used_rate,
    (rt_cv - AVG(rt_cv) OVER()) / NULLIF(stddev_samp(rt_cv) OVER(),0) AS z_rt_cv,
    (media_toggle_count - AVG(media_toggle_count) OVER()) / NULLIF(stddev_samp(media_toggle_count) OVER(),0) AS z_media_toggle_count,
    (quit_without_submit_rate - AVG(quit_without_submit_rate) OVER()) / NULLIF(stddev_samp(quit_without_submit_rate) OVER(),0) AS z_quit_without_submit_rate,
    (first_try_accuracy - AVG(first_try_accuracy) OVER()) / NULLIF(stddev_samp(first_try_accuracy) OVER(),0) AS z_first_try_accuracy,
    (self_correction_rate - AVG(self_correction_rate) OVER()) / NULLIF(stddev_samp(self_correction_rate) OVER(),0) AS z_self_correction_rate,
    (regret_rate - AVG(regret_rate) OVER()) / NULLIF(stddev_samp(regret_rate) OVER(),0) AS z_regret_rate
  FROM behavior
  JOIN rt_stats USING(user_id)
  JOIN quit_info USING(user_id)
  JOIN accuracy USING(user_id)
)

-- --------------------------------------------------
-- ⑧ 최종 요약 (30개 변수 + 종합지수)
-- --------------------------------------------------
SELECT
  b.user_id,

  -- 1️⃣ 기본 통계
  base.n_attempts, base.n_submit, base.n_quit, base.n_questions, base.n_bundles, base.active_days,

  -- 2️⃣ 시간·집중
  rts.rt_mean, rts.rt_std, rts.rt_cv, rts.consistent_study_rhythm,

  -- 3️⃣ 행동 패턴
  b.n_choice_changes, b.erasure_used, b.erasure_used_rate, b.media_toggle_count,

  -- 4️⃣ quit 관련
  q.quit_without_submit_rate,

  -- 5️⃣ 정답 관련
  a.first_try_accuracy, a.self_correction_rate, a.regret_rate, a.improvement_delta,

  -- 6️⃣ 학습 리듬·플랫폼
  rh.study_minutes_day, rh.start_hour_std, rh.platform_ratio_mobile, rh.platform_ratio_web,

  -- 7️⃣ 종합지수 (Z-score 기반)
  (
    z.z_n_choice_changes +
    z.z_erasure_used_rate +
    z.z_rt_cv +
    z.z_media_toggle_count +
    z.z_quit_without_submit_rate
  ) AS F_index,

  (
    z.z_first_try_accuracy +
    z.z_self_correction_rate -
    z.z_regret_rate -
    z.z_quit_without_submit_rate
  ) AS SE_index

FROM behavior b
JOIN base USING(user_id)
JOIN rt_stats rts USING(user_id)
JOIN quit_info q USING(user_id)
JOIN accuracy a USING(user_id)
JOIN rhythm rh USING(user_id)
JOIN z_norm z USING(user_id);
